# Workshop 2: Spatial & Geometry Operations on Vector data

Programación SIG  2025-1S    
Universidad Nacional de Colombia    
Alicia Arévalo Robinson

Workshop extracted from [Link](https://github.com/lccastillov/prog_sig_2025_1/blob/main/Workshop6_SpatialOpertaions_Vector_Data.ipynb)

### Modules

In [1]:
import geopandas as gpd
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import contextily as cx
from pathlib import Path
from shapely.geometry import Point

import numpy as np
import shapely
from shapely import affinity

### 1. Data

#### 1.1 Read files path

In [ ]:
# Main folder
root_folder = r"./data"

# Path to municipios_colombia.shp
shp_path = root_folder + "/municipios_colombia.shp"
# Read the shapefile using geopandas
muni = gpd.read_file(shp_path).to_crs(epsg=9377)

# Path to Evaluaciones_Agropecuarias_Municipales_EVA.csv
eva_path = root_folder + "/Evaluaciones_Agropecuarias_Municipales_EVA.csv"

# Path to Catalogo_Estaciones_IDEAM.csv"
stations_path = root_folder + "/Catalogo_Estaciones_IDEAM.csv"

# Path to precipitacion.csv"
precip_path = root_folder + "/precipitacion.csv"

DataSourceError: ./GIS_Programming_Data/municipios_colombia.shp: No such file or directory

#### 1.2 Read CSV files

In [ ]:
eva = pd.read_csv(eva_path, encoding='utf-8')
ideam = pd.read_csv(stations_path,encoding='utf-8')  
precip= pd.read_csv(precip_path, encoding='utf-8')

#### 1.3 Organize data

In [ ]:
# EVA table, rename code column
eva = (eva.rename(columns={"CÓD. MUN.": "cod_mun"}))
eva["CULTIVO"] = eva["CULTIVO"].astype(str)      # ensure string

#Homogenise municipalities codes
muni["MPIO_CCDGO"] = muni["MPIO_CCDGO"].astype(str).str.zfill(5)
eva["cod_mun"]     = eva["cod_mun"].astype(str).str.zfill(5)

print(f"{len(muni):,} municipalities • {len(ideam):,} stations • "
      f"{len(eva):,} EVA rows")

1,122 municipalities • 8,973 stations • 206,068 EVA rows


#### 1.4 Read `ideam` as a `GeoDataGrame`

In [ ]:
# Convert string coordinates to Points using string manipulation
ideam['geometry'] = ideam['Ubicación'].apply(
    lambda x: Point(
        float(x.strip('()').split(',')[1]),  # longitude
        float(x.strip('()').split(',')[0])   # latitude
    )
)

# Create GeoDataFrame
ideam_gdf = gpd.GeoDataFrame(
    ideam,
    geometry='geometry',
    crs="EPSG:4326"  # Standard lat/lon coordinates
)

# Reproject to match other layers
ideam_gdf = ideam_gdf.to_crs(9377)
ideam_gdf

,Codigo,Nombre,Categoria,Estado,Departamento,Municipio,Ubicación,Altitud,Fecha_instalacion,Fecha_suspension,Corriente,geometry
0,52057100,RUMICHACA - AUT [52057100],Limnigráfica,Activa,Nariño,Ipiales,"(0.81378611, -77.66197778)",2582,2016-11-15,NaN,Guaitara,POINT (4480921.418 1648261.853)
1,52055170,LA JOSEFINA - AUT [52055170],Climática Principal,Activa,Nariño,Contadero,"(0.93030556, -77.49119444)",2450,2005-12-10,NaN,Guaitara,POINT (4499992.408 1661153.867)
2,52055220,EL PARAISO - AUT [52055220],Climática Principal,Activa,Nariño,Túquerres,"(1.07061111, -77.63688889)",3120,2004-03-01,NaN,Guaitara,POINT (4483759.028 1676727.973)
3,44015070,EL PEPINO - AUT [44015070],Climática Principal,Activa,Putumayo,Mocoa,"(1.08288889, -76.66711111)",760,2005-11-11,NaN,Guaitara,POINT (4591897.597 1677940.612)
4,48015040,PUERTO NARINO - AUT [48015040],Climática Principal,Activa,Amazonas,Puerto Nariño,"(-3.78030556, -70.36263889)",158,2005-07-19,NaN,Amazonas,POINT (5292824.016 1139928.729)
...,...,...,...,...,...,...,...,...,...,...,...,...
8968,4401700167,NIVEL SANGOYACO GARGANTA,Limnimétrica,Activa,Putumayo,Mocoa,"(1.15872222, -76.67080556)",761,2020-02-25,NaN,Mocoa,POINT (4591496.422 1686336.979)
8969,4401700168,NIVEL MULATO PALMERAS,Limnimétrica,Activa,Putumayo,Mocoa,"(1.15638889, -76.69027778)",966,2020-02-25,NaN,Mocoa,POINT (4589326.12 1686081.462)
8970,4401700172,NIVEL MOCOA PISCIKART,Limnimétrica,Activa,Putumayo,Villagarzón,"(1.03964167, -76.61091944)",408,2020-02-25,NaN,Mocoa,POINT (4598153.817 1673145.302)
8971,4401700173,NIVEL RUMIYACO LAGARTO,Limnimétrica,Activa,Putumayo,Mocoa,"(1.119225, -76.67045)",647,2020-02-25,NaN,Mocoa,POINT (4591530.468 1681964.004)
